**제목**: Deep Learning Tutorial Using Keras Part03 - RNN (Keras로 배우는 딥러닝 튜토리얼 Part03 RNN편)<br>
**제작자**: Park Chanjun (박찬준)<br>
**소속**: Korea University Natural Language Processing & Artificial Intelligence Lab (고려대학교 자연언어처리&인공지능 연구실)<br>
**Email**: bcj1210@naver.com<br>
**참고자료**: 케라스 창시자에게 배우는 딥러닝

**텍스트와 시퀀스 데이터를 위한 딥러닝(자연언어처리)**
<br>

시퀀스 데이터를 처리하는 기본적인 딥러닝 모델은 <br>
1. Recurrent Neural Network 
2. 1D Convnet


크게 2가지가 존재한다.

딥러닝 모델은 수치형 텐서만 다룰 수 있다.<br>
따라서 텍스트를 수치형 텐서로 변환해야 한다.<br>
이 과정을 텍스트 벡터화(Vectorizing)이라고 한다.<br>
텍스트를 나누는 단위에(Token) 따라 크게 3가지 방식으로 이루어진다.

1. 텍스트를 단어로 나누고 각 단어를 하나의 벡터로 변환한다.
2. 텍스트를 문자로 나누고 각 문자를 하나의 벡터로 변환한다.
3. 텍스트에서 단어나 문자의 n-gram을 추출하여 n-gram을 하나의 벡터로 변환한다.
n-gram이란 문장에서 추출한 N개의 연속된 단어 그룹을 의미한다.

Token과 Vector를 연결하는 2가지 방법
1. One Hot Encoding
2. (Word or Char or Sentence or Document) Embedding<br>
a. 문제와 함께 단어 임베딩을 학습, 랜덤한 단어 벡터로 시작하여 신경망의 가중치를 학습하는 것과 같은 방식으로 단어 벡터를 학습<br>
b. 사전 훈련된 단어 임베딩(Pretrain word Embedding)사용<br>

![대체 텍스트](https://miro.medium.com/max/2560/1*52X2L01wpUjy39lIjofC7g.jpeg)




In [0]:
#One-Hot Encoding 실습 (단어 수준)

import numpy as np

samples=["안녕하세요 저의 이름은 박찬준입니다.","지금부터 자연언어처리 강의를 시작하도록 하겠습니다."] #샘플 데이터

token_index={}#토큰 딕셔너리

for sample in samples:
  for word in sample.split():#split을 이용하여 token으로 분리
    if word not in token_index:
      token_index[word]=len(token_index)+1 #단어마다 고유한 인덱스 부여, 인덱스 0은 사용하지 않기에 +1 을 하였음.

max_length=10

results=np.zeros(shape=(len(samples),
                        max_length,
                        max(token_index.values())+1))#결과를 저장할 배열

for i,sample in enumerate(samples):
  for j,word in list(enumerate(sample.split()))[:max_length]:
    index=token_index.get(word)
    results[i,j,index]=1.

print(results)



In [0]:
#One-Hot Encoding 실습 (문자 수준)
import numpy as np
import string

samples=["안녕하세요 저의 이름은 박찬준입니다.","지금부터 자연언어처리 강의를 시작하도록 하겠습니다."] #샘플 데이터
characters=string.printable #출력가능한 ASCII 문자

token_index=dict(zip(characters, range(1,len(characters)+1))) #char 딕셔너리 생성

max_length=50

results=np.zeros(len(samples),max_length,max(token_index.values()+1))

for i,sample in enumerate(samples):
  for i,character in enumerate(sample):
    index=token_index.get(character)
    results[i,j,index]=1.

In [0]:
#One-Hot Encoding 실습 (Keras  이용)
from keras.preprocessing.text import Tokenizer

samples=["안녕하세요 저의 이름은 박찬준입니다.","안녕하세요 지금부터 자연언어처리 강의를 시작하도록 하겠습니다."] #샘플 데이터

tokenizer=Tokenizer(num_words=1000)#빈도수가 높은 1000개의 단어만 선택.
tokenizer.fit_on_texts(samples)#단어 인덱스 구축

sequences=tokenizer.texts_to_sequences(samples)#문자열을 정수 인덱스 리스트로 변환
print("sequences: ",sequences)

one_hot_results=tokenizer.texts_to_matrix(samples,mode='binary')#직접 one hot 이진 벡터 표현을 얻을 수 있다.
print("one_hot_results: ",one_hot_results)

word_index=tokenizer.word_index #단어 인덱스
print("word_index: ",word_index)

embedding_layer=Embedding(1000,64) #가능한 토큰의 개수(단어 인덱스 최대값+1), 임베딩 차원
1. Embedding 층은 간단히 생각해 정수 인덱스를 밀집 벡터로 매핑하는 딕셔너리이다.
2. 단어 인덱스 => Embedding층 => 연관된 단어 벡터
3. 배치에 있는 모든 시퀀스는 길이가 같아야 하므로 작은 길이의 시퀀스는 0으로 패딩된다.

4. Embedding 층은 (samples,sequence_length) 인 2D 정수 텐서를 입력으로 받는다.
5. Embedding 층은 (sampels,sequence_length,embedding_dimensionality)인 3D 실수형 텐서를 반환한다.

In [0]:
#Word Embedding 실습  => 문제와 함께 단어 임베딩을 학습, 랜덤한 단어 벡터로 시작하여 신경망의 가중치를 학습하는 것과 같은 방식으로 단어 벡터를 학습
#저차원의 실수벡터 (보통 256,512,1024차원 정도 사용)

from keras.layers import Embedding
from keras.datasets import imdb
from keras import preprocessing
from keras.models import Sequential
from keras.layers import Flatten,Dense,Embedding

max_features=10000 #특성으로 사용할 단어의 수
maxlen=20  #사용할 텍스트 길이

#데이터 로딩
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=max_features)

#패딩
x_train=preprocessing.sequence.pad_sequences(x_train,maxlen=maxlen) #패딩 , (sampels,maxlen)의 2D 텐서 반환
x_test=preprocessing.sequence.pad_sequences(x_test,maxlen=maxlen) #패딩 , (sampels,maxlen)의 2D 텐서 반환

#모델 생성
model=Sequential()
model.add(Embedding(10000, 8, input_length=maxlen)) #출력은 (samples,maxlen,8)

model.add(Flatten()) #(samples,maxlen*8)의 2D 텐서로 펼친다.

model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

model.summary()

history=model.fit(x_train,y_train,epochs=10,batch_size=32,validation_split=0.2)

test_loss,test_acc=model.evaluate(x_test,y_test)
print("Test_acc: ",test_acc)

먼저 http://mng.bz/0tIo 에서 IMDB 원본 데이터셋을 다운로드하고 압축을 해제합니다.

In [0]:
import os

imdb_dir = './datasets/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

#훈련용 리뷰 하나를 문자열 하나로 만들어 훈련 데이터를 문자열의 리스트로 구성. 리뷰 레이블(긍정/부정)도 labels 리스트로 만듬.
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding='utf8')
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('%s개의 고유한 토큰을 찾았습니다.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('데이터 텐서의 크기:', data.shape)
print('레이블 텐서의 크기:', labels.shape)

# 데이터를 훈련 세트와 검증 세트로 분할합니다.
# 샘플이 순서대로 있기 때문에 (부정 샘플이 모두 나온 후에 긍정 샘플이 옵니다) 
# 먼저 데이터를 섞습니다.
indices = np.arange(data.shape[0])
np.random.shuffle(indices) #셔플링 
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

GloVe 단어 임베딩 내려받기
https://nlp.stanford.edu/projects/glove 에서 2014년 영문 위키피디아를 사용해 사전에 계산된 임베딩을 내려받습니다. 이 파일의 이름은 glove.6B.zip이고 압축 파일 크기는 823MB입니다. 400,000만개의 단어(또는 단어가 아닌 토큰)에 대한 100차원의 임베딩 벡터를 포함하고 있습니다. datasets 폴더 아래에 파일 압축을 해제합니다.(이 저장소에는 이미 포함되어 있습니다)

In [0]:
glove_dir = './datasets/'

#Glove 처리 
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values[0] #단어
    coefs = np.asarray(values[1:], dtype='float32') #벡터값
    embeddings_index[word] = coefs #딕셔너리
f.close()

print('%s개의 단어 벡터를 찾았습니다.' % len(embeddings_index))


#Embedding층에 주입할 수 있는 임베딩 행렬을 만들어야 한다.
#행렬의 크기는 (max_words,embedding_dim)이어야 한다.
embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # 임베딩 인덱스에 없는 단어는 모두 0이 됩니다.
            embedding_matrix[i] = embedding_vector

from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

#모델 정의
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

#모델에 GloVe 임베딩 로드하기
#Embedding 층은 하나의 가중치 행렬을 가집니다. 이 행렬은 2D 부동 소수 행렬이고 각 i번째 원소는 i번째 인덱스에 상응하는 단어 벡터입니다. 모델의 첫 번째 층인 Embedding 층에 준비된 GloVe 행렬을 로드
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False #추가적으로 Embedding 층을 동결합니다

#모델 컴파일
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

#모델 훈련
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
#모델 저장
model.save_weights('pre_trained_glove_model.h5')

#테스트 
test_dir = os.path.join(imdb_dir, 'test')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in sorted(os.listdir(dir_name)):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding="utf8")
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)

model.load_weights('pre_trained_glove_model.h5')
model.evaluate(x_test, y_test)

**RNN**

![대체 텍스트](http://www.wildml.com/wp-content/uploads/2015/09/rnn.jpg)

<br>
RNN은 (batch_size, timesteps,input_features)인 2D 텐서로 인코딩 된 벡터의 시퀀스를 입력 받음.

In [0]:
from keras.datasets import imdb
from keras.preprocessing import sequence

from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

max_features = 10000  # 특성으로 사용할 단어의 수
maxlen = 500  # 사용할 텍스트의 길이(가장 빈번한 max_features 개의 단어만 사용합니다)
batch_size = 32

#데이터 로딩
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)

#패딩
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print('input_train 크기:', input_train.shape)
print('input_test 크기:', input_test.shape)

from keras.layers import Dense

#모델 정의 
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

#모델 컴파일
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

#모델 훈련
history = model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

import matplotlib.pyplot as plt

#시각화 
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()



**LSTM과 GRU**
<br>
Simple RNN의 문제점은 Vanishing Gradient Problem 즉 기울기 소실 문제.<br>
이 문제를 해결하기 위한 것이 LSTM,GRU이다<br>
과거 정보를 나중에 다시 주입하여 기울기 소실 문제를 해결하려 하는 것 !

![대체 텍스트](https://miro.medium.com/max/3032/1*yBXV9o5q7L_CvY7quJt3WQ.png)

In [0]:
from keras.datasets import imdb
from keras.preprocessing import sequence

from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

max_features = 10000  # 특성으로 사용할 단어의 수
maxlen = 500  # 사용할 텍스트의 길이(가장 빈번한 max_features 개의 단어만 사용합니다)
batch_size = 32

#데이터 로딩
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)

#패딩
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print('input_train 크기:', input_train.shape)
print('input_test 크기:', input_test.shape)


#모델 정의 
from keras.layers import Dense
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32)) #LSTM 사용 !
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)



import matplotlib.pyplot as plt

#시각화 
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


**RNN 고급 사용법**

1. 순환 드롭 아웃: Recurrent Dropout: 순환층에 과대적합 방지
2. 스태킹 순환 층: Stacking Recurrent Layer: 네트워크 표현 능력 증가시킴
3. Bidirectional Recurrent Layer: 양방향

In [0]:
import os

#데이터 받아오기 
!mkdir ./datasets
!mkdir ./datasets/jena_climate/
!wget https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
!unzip jena_climate_2009_2016.csv.zip
!mv jena_climate_2009_2016.csv ./datasets/jena_climate/

#데이터 로딩
data_dir = './datasets/jena_climate/'
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv')

f = open(fname)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

print(header)
print(len(lines))

#데이터 파싱 
import numpy as np

float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values

#정규화
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std

#제너레이터 생성 
def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows),
                           lookback // step,
                           data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets

lookback = 1440
step = 6
delay = 144
batch_size = 128

train_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=200000,
                      shuffle=True,
                      step=step, 
                      batch_size=batch_size)
val_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=200001,
                    max_index=300000,
                    step=step,
                    batch_size=batch_size)
test_gen = generator(float_data,
                     lookback=lookback,
                     delay=delay,
                     min_index=300001,
                     max_index=None,
                     step=step,
                     batch_size=batch_size)

# 전체 검증 세트를 순회하기 위해 val_gen에서 추출할 횟수
val_steps = (300000 - 200001 - lookback) // batch_size

# 전체 테스트 세트를 순회하기 위해 test_gen에서 추출할 횟수
test_steps = (len(float_data) - 300001 - lookback) // batch_size


def evaluate_naive_method():
    batch_maes = []
    for step in range(val_steps):
        samples, targets = next(val_gen)
        preds = samples[:, -1, 1]
        mae = np.mean(np.abs(preds - targets))
        batch_maes.append(mae)
    print(np.mean(batch_maes))
    
evaluate_naive_method()


#GRU 사용
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.GRU(32, input_shape=(None, float_data.shape[-1])))#GRU
model.add(layers.Dense(1))

model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=20,
                              validation_data=val_gen,
                              validation_steps=val_steps)




#드롭아웃 적용해보기 
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.GRU(32,
                     dropout=0.2, #드롭아웃
                     recurrent_dropout=0.2, #드롭아웃 
                     input_shape=(None, float_data.shape[-1])))
model.add(layers.Dense(1))

model.compile(optimizer=RMSprop(), loss='mae')

history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=40,
                              validation_data=val_gen,
                              validation_steps=val_steps)

#스태킹 순환층
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.GRU(32,
                     dropout=0.1,
                     recurrent_dropout=0.5,
                     return_sequences=True,#요렇게 설정하면됨 !!!!!!!!!!!1
                     input_shape=(None, float_data.shape[-1])))

model.add(layers.GRU(64, activation='relu',
                     dropout=0.1, 
                     recurrent_dropout=0.5))
model.add(layers.Dense(1))

model.compile(optimizer=RMSprop(), loss='mae')

history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=40,
                              validation_data=val_gen,
                              validation_steps=val_steps)



#거꾸로 된 시퀀스를 가지고 훈련해보기 
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras import layers
from keras.models import Sequential

# 특성으로 사용할 단어의 수
max_features = 10000
# 사용할 텍스트의 길이(가장 빈번한 max_features 개의 단어만 사용합니다)
maxlen = 500

# 데이터 로드
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# 시퀀스를 뒤집습니다!!!!!!!!!!!!!!!!!!!!!1
x_train = [x[::-1] for x in x_train]
x_test = [x[::-1] for x in x_test]

# 시퀀스에 패딩을 추가합니다
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

model = Sequential()
model.add(layers.Embedding(max_features, 128))
model.add(layers.LSTM(32))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)


#양방향 LSTM
from keras import backend as K
K.clear_session()

model = Sequential()
model.add(layers.Embedding(max_features, 32))
model.add(layers.Bidirectional(layers.LSTM(32)))#Bidirectional!!!!!!!!!
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)


from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.Bidirectional(
    layers.GRU(32), input_shape=(None, float_data.shape[-1])))
model.add(layers.Dense(1))

model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=40,
                              validation_data=val_gen,
                              validation_steps=val_steps)




**CNN을 이용한 시퀀스 처리**
<br>
![대체 텍스트](http://i.imgur.com/JN72JHW.png)

In [0]:
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000  # 특성으로 사용할 단어의 수
max_len = 500  # 사용할 텍스트의 길이(가장 빈번한 max_features 개의 단어만 사용합니다)

#데이터 로딩
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)


#패딩
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)
print('x_train 크기:', x_train.shape)
print('x_test 크기:', x_test.shape)

from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

#모델 정의 
model = Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))

#모델 출력
model.summary()

#모델 컴파일
model.compile(optimizer=RMSprop(lr=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])

#모델 훈련
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)


test_loss,test_acc=model.evaluate(x_test,y_test)
print("Test_acc: ",test_acc)


**CNN과 RNN 연결하여 긴 시퀀스 처리하기**


In [0]:
import os
import numpy as np

data_dir = './datasets/jena_climate/'
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv')

f = open(fname)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values
    
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std

def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows),
                           lookback // step,
                           data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets
        
lookback = 1440
step = 6
delay = 144
batch_size = 128

train_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=200000,
                      shuffle=True,
                      step=step, 
                      batch_size=batch_size)

val_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=200001,
                    max_index=300000,
                    step=step,
                    batch_size=batch_size)

test_gen = generator(float_data,
                     lookback=lookback,
                     delay=delay,
                     min_index=300001,
                     max_index=None,
                     step=step,
                     batch_size=batch_size)

# 전체 검증 세트를 순회하기 위해 val_gen에서 추출할 횟수
val_steps = (300000 - 200001 - lookback) // batch_size

# 전체 테스트 세트를 순회하기 위해 test_gen에서 추출할 횟수
test_steps = (len(float_data) - 300001 - lookback) // batch_size

from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

#모델 정의
model = Sequential()

#Conv 1D
model.add(layers.Conv1D(32, 5, activation='relu',
                        input_shape=(None, float_data.shape[-1])))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))

#GRU
model.add(layers.GRU(32, dropout=0.1, recurrent_dropout=0.5))
model.add(layers.Dense(1))

model.summary()

model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=20,
                              validation_data=val_gen,
                              validation_steps=val_steps)

